# Historical Flood Data

In [ ]:
import pandas as pd
# historical flood data
def clean_flood_data():
    flood_df = pd.read_excel('FloodArchive.xlsx', engine='openpyxl')

    #filter only for United States
    flood_df = flood_df[flood_df['Country'] == 'USA']
    #add a zip code based on the long and lat

    return flood_df

flood_df = clean_flood_data()

FileNotFoundError: [Errno 2] No such file or directory: 'FloodArchive.xlsx'

In [ ]:
print(f'USA number of flood incidents: {len(flood_df)}')
print(flood_df.head())
#save as csv
flood_df.to_csv('united_states_floods.csv', index=False)


In [ ]:
# !pip install folium
import folium
import math

#visual representation of FLOOD DATA
map_usa = folium.Map(location=[39.8283, -98.5795], zoom_start=5)
severity_colors = {
    1.0: "yellow", #large flood events: 1-2 decades-long reported interval since the last similar event
    1.5: "orange", #very large events: greater than 2 decades but less than 100 year estimated recurrence interval
    2.0: "red" #Extreme events: with an estimated recurrence interval greater than 100 years.
}

# Add a marker for each flood occurrence
for index, row in flood_df.iterrows():
    # Add a circle marker at each flood location (lat, long)
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        popup=f"ID: {row['ID']} | Severity: {row['Severity']} | Displaced: {row['Displaced']} | Date: {row['Began']}",
        color=severity_colors[row['Severity']] ,
        fill=True,
        fill_opacity=0.6
    ).add_to(map_usa)

# Save map to HTML file
map_usa.save('flood_map.html')


In [ ]:
!pip install sqlalchemy_mate==2.0.0.0

#add zipcode column to flood data
from uszipcode import SearchEngine

search = SearchEngine()

def get_zipcode(lat, lon):
    result = search.by_coordinates(lat, lon)
    if result:
        return result[0].zipcode
    return None

# Apply the get_zipcode function to each row and create a new column 'zipcode'
flood_df['zipcode'] = flood_df.apply(lambda row: get_zipcode(row['lat'], row['long']), axis=1)


In [ ]:
#save as csv
pd.set_option('display.max_columns', None)
print(flood_df.head())
flood_df.to_csv('united_states_floods.csv', index=False)

#Historical Housing Data

In [ ]:
valid_zipcodes = flood_df[~flood_df['zipcode'].isna()]['zipcode'].unique()
def clean_housing_data():
    df = pd.read_csv('hpi_at_bdl_zip5.csv', dtype={'Five-Digit ZIP Code': str})
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #zipcodes from flood data
    df = df[df['Five-Digit ZIP Code'].isin(valid_zipcodes)]
    return df

df_housing = clean_housing_data()

In [ ]:
print(df_housing.head(10))
#save as csv
df_housing.to_csv('united_states_housing.csv', index=False)


     Five-Digit ZIP Code  Year  Annual Change (%)     HPI  HPI with 1990 base  \
2401               01253  2001                NaN  100.00                 NaN   
2402               01253  2002               9.38  109.38                 NaN   
2403               01253  2003               2.17  111.75                 NaN   
2404               01253  2004              21.80  136.12                 NaN   
2405               01253  2005               8.09  147.13                 NaN   
2406               01253  2006              12.92  166.14                 NaN   
2407               01253  2007              -2.73  161.60                 NaN   
2408               01253  2008              -1.61  159.00                 NaN   
2409               01253  2009              -2.66  154.77                 NaN   
2410               01253  2010               0.33  155.28                 NaN   

      HPI with 2000 base  
2401                 NaN  
2402                 NaN  
2403                 NaN  


#Random Forest Regressor

In [48]:
#code for predicing HPI based on flood data
#load data
import pandas as pd
flood_data = pd.read_csv('united_states_floods.csv', dtype={'zipcode': str})
housing_data = pd.read_csv('united_states_housing.csv', dtype={'Five-Digit ZIP Code': str})
print(flood_data.head(5))


   ID GlideNumber Country OtherCountry      long      lat       Area  \
0   9           0     USA            0  -85.1742  40.6691  210527.96   
1  11           0     USA            0  -89.5537  40.6814   26266.14   
2  12           0     USA            0 -108.0930  35.3824   26527.13   
3  13           0     USA            0  -96.7845  29.6044  141508.00   
4  14           0     USA            0  -83.5377  42.0122   16883.54   

        Began       Ended Validation  Dead  Displaced          MainCause  \
0  1985-02-22  1985-03-01       News     7       2250  Rain and snowmelt   
1  1985-03-03  1985-03-08       News     4       2400  Rain and snowmelt   
2  1985-03-13  1985-03-14       News     0         80  Rain and snowmelt   
3  1985-03-14  1985-03-15       News     0          0         Heavy rain   
4  1985-03-30  1985-03-31       News     0        300         Heavy rain   

   Severity zipcode  
0       2.0   46781  
1       2.0   61611  
2       1.0   87045  
3       1.0   78962  


In [49]:
print(housing_data.head(5))

  Five-Digit ZIP Code  Year  Annual Change (%)     HPI  HPI with 1990 base  \
0               01253  2001                NaN  100.00                 NaN   
1               01253  2002               9.38  109.38                 NaN   
2               01253  2003               2.17  111.75                 NaN   
3               01253  2004              21.80  136.12                 NaN   
4               01253  2005               8.09  147.13                 NaN   

   HPI with 2000 base  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  


In [50]:
#SUMMARIZE FLOOD EVENTS
# For each zip code, get a summary of the floods that occurred up to and including that year.
# calculate duration(in days)
flood_data['Began'] = pd.to_datetime(flood_data['Began'])
flood_data['Ended'] = pd.to_datetime(flood_data['Ended'])
flood_data.loc[:, 'Duration'] = (flood_data['Ended'] - flood_data['Began']).dt.days
flood_data['Year'] = flood_data['Began'].dt.year


# # aggregate flood data by zip code and year
flood_summary = flood_data.groupby(['zipcode', 'Year']).agg(
    flood_count=('Severity', 'count'),
    max_severity=('Severity', lambda x: x.max()),
    median_dead=('Dead', 'median'),
    median_displaced=('Displaced', 'median'),
    median_duration=('Duration', 'median'),
    median_area = ('Area', 'median')
).reset_index()

flood_summary



,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area
0,00641,1985,1,1.0,2.0,4500.0,1.0,1423.92
1,00664,1985,1,1.0,99.0,6200.0,2.0,7219.80
2,00698,2001,1,1.0,2.0,0.0,1.0,245.95
3,01031,1996,1,2.0,5.0,700.0,4.0,67013.98
4,01031,2007,1,1.0,9.0,0.0,6.0,109079.09
...,...,...,...,...,...,...,...,...
450,99705,1991,1,1.0,2.0,800.0,4.0,22130.06
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67
452,99709,2003,1,1.0,0.0,0.0,5.0,6718.98
453,99743,2003,1,1.0,0.0,20.0,4.0,13111.55


In [51]:
housing_data.rename(columns={'Five-Digit ZIP Code': 'zipcode'}, inplace=True)

# Merge flood summary with housing price index data based on 'zipcode' and 'Year'
merged_data = pd.merge(flood_summary, housing_data, on=['zipcode', 'Year'], how='left')

# Display the first few rows of the merged data
merged_data

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,00641,1985,1,1.0,2.0,4500.0,1.0,1423.92,NaN,NaN,NaN,NaN
1,00664,1985,1,1.0,99.0,6200.0,2.0,7219.80,NaN,NaN,NaN,NaN
2,00698,2001,1,1.0,2.0,0.0,1.0,245.95,NaN,NaN,NaN,NaN
3,01031,1996,1,2.0,5.0,700.0,4.0,67013.98,NaN,NaN,NaN,NaN
4,01031,2007,1,1.0,9.0,0.0,6.0,109079.09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
450,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,20.76,72.44,120.76,69.72
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,4.43,113.15,188.63,108.90
452,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,2.88,246.91,162.72,110.30
453,99743,2003,1,1.0,0.0,20.0,4.0,13111.55,NaN,NaN,NaN,NaN


In [53]:
#drop unmatched rows
#NOTE ROWS DROPPED PRETTY SIGNIFICANTLY
merged_data = merged_data.dropna(subset=['HPI'])
merged_data.to_csv('merged_housing_flood_data.csv', index=False)
merged_data

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
6,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,6.20,468.17,230.13,164.19
8,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,0.50,122.27,NaN,NaN
12,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,3.83,314.67,214.37,164.46
18,05680,2002,1,1.0,0.0,0.0,2.0,11377.32,5.12,105.12,NaN,NaN
19,06023,1992,1,1.0,0.0,250.0,1.0,4609.14,NaN,100.00,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...
448,99669,2002,1,2.0,0.0,0.0,4.0,13912.96,1.72,116.20,NaN,107.77
450,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,20.76,72.44,120.76,69.72
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,4.43,113.15,188.63,108.90
452,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,2.88,246.91,162.72,110.30


In [54]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score


#Use RandomForestRegressor to predict HPI based on floods

X = merged_data[['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area']]
y = merged_data['HPI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# evaluate the model performance
print("Random Forest Regressor: Predicting HPI(no past history, just current year):")
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

relative_error = mae / np.mean(y_test) * 100
print(f'Relative Error: {relative_error:.2f}%')

print(f"Predictions: {y_pred[:10]}")
print(f"Actual: {y_test[:10].values}")

Random Forest Regressor: Predicting HPI(no past history, just current year):
R² score: 0.038
Mean Absolute Error (MAE): 129.130
Relative Error: 49.68%
Predictions: [164.2171 293.2571 211.43   176.737  163.9115 169.1713 186.2824 291.9731
 377.9636 205.0252]
Actual: [ 147.38 2220.62  236.46  186.07  174.23  113.15  179.46  733.52  127.62
  352.21]


#### Improved Random Forest Regressor(w/ hyperparameter tuning)



In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np

X = merged_data[['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area']]
y = merged_data['HPI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


rf_model = RandomForestRegressor(random_state=42)

param_dist = {
    'n_estimators': np.arange(50, 301, 50),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'bootstrap': [True, False],
}

random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, random_state=42, verbose=2)
random_search.fit(X_train_scaled, y_train)
print(f"Best parameters from RandomizedSearchCV: {random_search.best_params_}")

best_rf_model = random_search.best_estimator_
best_rf_model.fit(X_train_scaled, y_train)
y_pred = best_rf_model.predict(X_test_scaled)

# evaluate the model's performance
print("Random Forest Regressor: Predicting HPI(no past history, just current year):")
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

relative_error = mae / np.mean(y_test) * 100
print(f'Relative Error: {relative_error:.2f}%')

print(f"Predictions: {y_pred[:10]}")  # First 10 predicted values
print(f"Actual: {y_test[:10].values}")  # First 10 actual values

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters from RandomizedSearchCV: {'n_estimators': np.int64(100), 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_depth': 40, 'bootstrap': True}
Random Forest Regressor: Predicting HPI(no past history, just current year):
R² score: 0.058
Mean Absolute Error (MAE): 124.704
Relative Error: 47.98%
Predictions: [171.93103007 321.14953436 212.39539578 184.54486418 171.93103007
 190.71014019 183.10640241 267.25928276 260.11102714 217.6068887 ]
Actual: [ 147.38 2220.62  236.46  186.07  174.23  113.15  179.46  733.52  127.62
  352.21]


### Random Forest Regressor w/ past memory(10 years prior)

In [56]:
flood_data = flood_data.dropna(subset=['zipcode', 'Year'])

In [57]:
# Create the 3-area zipcode column (first 3 digits of zipcode)
flood_summary['zipcode_3area'] = flood_summary['zipcode'].astype(str).str[:3]
flood_summary = flood_summary.sort_values(by=['zipcode_3area', 'Year'])

flood_summary_10_years = flood_summary.copy()

# Rolling average for each feature, grouped by the 3-area zipcode
flood_summary_10_years['flood_count_total_10'] = flood_summary.groupby('zipcode_3area')['flood_count'].transform(lambda x: x.rolling(window=10, min_periods=1).sum())
flood_summary_10_years['max_severity_avg_10'] = flood_summary.groupby('zipcode_3area')['max_severity'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())
flood_summary_10_years['median_dead_avg_10'] = flood_summary.groupby('zipcode_3area')['median_dead'].transform(lambda x: x.rolling(window=10, min_periods=1).median())
flood_summary_10_years['median_displaced_avg_10'] = flood_summary.groupby('zipcode_3area')['median_displaced'].transform(lambda x: x.rolling(window=10, min_periods=1).median())
flood_summary_10_years['median_duration_avg_10'] = flood_summary.groupby('zipcode_3area')['median_duration'].transform(lambda x: x.rolling(window=10, min_periods=1).median())
flood_summary_10_years['median_area_avg_10'] = flood_summary.groupby('zipcode_3area')['median_area'].transform(lambda x: x.rolling(window=10, min_periods=1).median())


flood_summary_10_years.head(10)


,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,zipcode_3area,flood_count_total_10,max_severity_avg_10,median_dead_avg_10,median_displaced_avg_10,median_duration_avg_10,median_area_avg_10
0,00641,1985,1,1.0,2.0,4500.0,1.0,1423.92,006,1.0,1.0,2.0,4500.0,1.0,1423.920
1,00664,1985,1,1.0,99.0,6200.0,2.0,7219.80,006,2.0,1.0,50.5,5350.0,1.5,4321.860
2,00698,2001,1,1.0,2.0,0.0,1.0,245.95,006,3.0,1.0,2.0,4500.0,1.0,1423.920
3,01031,1996,1,2.0,5.0,700.0,4.0,67013.98,010,1.0,2.0,5.0,700.0,4.0,67013.980
4,01031,2007,1,1.0,9.0,0.0,6.0,109079.09,010,2.0,1.5,7.0,350.0,5.0,88046.535
5,01253,1998,1,2.0,0.0,0.0,2.0,1697.01,012,1.0,2.0,0.0,0.0,2.0,1697.010
6,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,017,1.0,1.5,0.0,0.0,2.0,5095.450
7,03262,1990,1,1.0,0.0,400.0,4.0,20288.60,032,1.0,1.0,0.0,400.0,4.0,20288.600
8,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,034,1.0,2.0,11.0,3000.0,9.0,38291.770
10,03585,1998,1,2.0,21.0,11000.0,7.0,18116.79,035,1.0,2.0,21.0,11000.0,7.0,18116.790


In [58]:
merged_data = pd.merge(flood_summary_10_years, housing_data, on=['zipcode', 'Year'], how='left')
merged_data = merged_data.dropna(subset=['HPI'])

#save data
merged_data.to_csv('merged_housing_flood_data_avg_10_years.csv', index=False)
merged_data

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,zipcode_3area,flood_count_total_10,max_severity_avg_10,median_dead_avg_10,median_displaced_avg_10,median_duration_avg_10,median_area_avg_10,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
6,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,017,1.0,1.5,0.0,0.0,2.0,5095.450,6.20,468.17,230.13,164.19
8,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,034,1.0,2.0,11.0,3000.0,9.0,38291.770,0.50,122.27,NaN,NaN
12,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,039,1.0,2.0,1.0,2500.0,11.0,42910.990,3.83,314.67,214.37,164.46
18,05680,2002,1,1.0,0.0,0.0,2.0,11377.32,056,2.0,1.5,0.0,0.0,2.0,6344.380,5.12,105.12,NaN,NaN
19,06023,1992,1,1.0,0.0,250.0,1.0,4609.14,060,1.0,1.0,0.0,250.0,1.0,4609.140,NaN,100.00,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,99669,2002,1,2.0,0.0,0.0,4.0,13912.96,996,2.0,2.0,0.0,150.0,16.0,32151.255,1.72,116.20,NaN,107.77
449,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,997,1.0,1.0,2.0,800.0,4.0,22130.060,20.76,72.44,120.76,69.72
451,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,997,3.0,1.0,0.0,0.0,4.0,1681.670,4.43,113.15,188.63,108.90
452,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,997,4.0,1.0,0.0,0.0,4.0,4200.325,2.88,246.91,162.72,110.30


In [59]:
X = merged_data[['flood_count', 'max_severity', 'median_dead', 'median_displaced',
                 'median_duration', 'median_area',
                 'flood_count_total_10', 'max_severity_avg_10',
                 'median_dead_avg_10', 'median_displaced_avg_10',
                 'median_duration_avg_10', 'median_area_avg_10']]

y = merged_data['HPI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_split= 15, min_samples_leaf= 1, max_depth= 40, bootstrap= True)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# evaluate the model's performance
print("Random Forest Regressor: Predicting HPI(w/ past history, prior 10 year flood averages):")
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

relative_error = mae / np.mean(y_test) * 100
print(f'Relative Error: {relative_error:.2f}%')

print(f"Predictions: {y_pred[:10]}")  # First 10 predicted values
print(f"Actual: {y_test[:10].values}")  # First 10 actual values

Random Forest Regressor: Predicting HPI(w/ past history, prior 10 year flood averages):
R² score: 0.120
Mean Absolute Error (MAE): 131.857
Relative Error: 49.43%
Predictions: [172.68589823 403.84658006 238.97032686 287.32606985 228.25698646
 208.59917858 176.80587846 239.45788787 316.13303172 264.96035978]
Actual: [ 147.38 2220.62  236.46  186.07  128.93  113.15  179.46  733.52  127.62
  352.21]


# Synthesizing Data

## Current Year Synthesized

In [60]:
import pandas as pd
original_data = pd.read_csv('merged_housing_flood_data.csv', dtype={'zipcode': str})
original_data

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,6.20,468.17,230.13,164.19
1,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,0.50,122.27,NaN,NaN
2,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,3.83,314.67,214.37,164.46
3,05680,2002,1,1.0,0.0,0.0,2.0,11377.32,5.12,105.12,NaN,NaN
4,06023,1992,1,1.0,0.0,250.0,1.0,4609.14,NaN,100.00,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...
207,99669,2002,1,2.0,0.0,0.0,4.0,13912.96,1.72,116.20,NaN,107.77
208,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,20.76,72.44,120.76,69.72
209,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,4.43,113.15,188.63,108.90
210,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,2.88,246.91,162.72,110.30


In [61]:
from sklearn.mixture import GaussianMixture

from sklearn.metrics import pairwise_distances_argmin_min

numeric_columns = ['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area', 'HPI']
data_for_gmm = original_data[numeric_columns].dropna()

gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(data_for_gmm)

synthetic_data = gmm.sample(n_samples=1000)
synthetic_df = pd.DataFrame(synthetic_data[0], columns=numeric_columns)

distances = pairwise_distances_argmin_min(synthetic_df[numeric_columns], original_data[numeric_columns])
synthetic_df['zipcode'] = original_data['zipcode'].iloc[distances[0]].values
synthetic_df['Year'] = original_data['Year'].iloc[distances[0]].values

expanded_data = pd.concat([original_data, synthetic_df], axis=0)
expanded_data = expanded_data.reset_index(drop=True)


expanded_data


/usr/local/lib/python3.11/dist-packages/sklearn/mixture/_base.py:436: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1.000000,1.500000,0.000000,0.000000e+00,2.000000,5095.450000,6.20,468.170000,230.13,164.19
1,03448,2005,1.000000,2.000000,11.000000,3.000000e+03,9.000000,38291.770000,0.50,122.270000,NaN,NaN
2,03903,2006,1.000000,2.000000,1.000000,2.500000e+03,11.000000,42910.990000,3.83,314.670000,214.37,164.46
3,05680,2002,1.000000,1.000000,0.000000,0.000000e+00,2.000000,11377.320000,5.12,105.120000,NaN,NaN
4,06023,1992,1.000000,1.000000,0.000000,2.500000e+02,1.000000,4609.140000,NaN,100.000000,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...
1207,28305,2018,1.000030,2.000134,12.001981,1.500000e+06,17.000230,95203.219670,NaN,262.809246,NaN,NaN
1208,28305,2018,0.999888,1.999734,12.000183,1.500000e+06,17.001955,95203.219325,NaN,262.809618,NaN,NaN
1209,28305,2018,0.999147,2.000189,12.001521,1.500000e+06,17.000299,95203.221935,NaN,262.809282,NaN,NaN
1210,28305,2018,0.999589,2.000448,11.999018,1.500000e+06,16.999766,95203.218389,NaN,262.809165,NaN,NaN


In [62]:
X = expanded_data[['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area']]
y = expanded_data['HPI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_split= 15, min_samples_leaf= 1, max_depth= 40, bootstrap= True)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# evaluate the model performance
print("Random Forest Regressor: Predicting HPI(no past history, synthetic data):")
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')


mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

relative_error = mae / np.mean(y_test) * 100
print(f'Relative Error: {relative_error:.2f}%')



print(f"Predictions: {y_pred[:10]}")
print(f"Actual: {y_test[:10].values}")



Random Forest Regressor: Predicting HPI(no past history, synthetic data):
R² score: 0.539
Mean Absolute Error (MAE): 88.939
Relative Error: 39.31%
Predictions: [229.20967117 186.46057829 291.52535315 134.06142195 200.53251506
 217.42749197 285.86011346 171.44387813 230.08356007 235.83929783]
Actual: [204.28041764 142.80559294 162.32       225.2737102  139.71
 208.92421544 230.69017477 246.91       194.99734347 119.69913289]


## Prior Year Averages Synthesized

In [ ]:
import numpy as np
original_data_10 = pd.read_csv('merged_housing_flood_data_avg_10_years.csv', dtype={'zipcode': str, 'zipcode_3area':str})
original_data_10

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,zipcode_3area,flood_count_total_10,max_severity_avg_10,median_dead_avg_10,median_displaced_avg_10,median_duration_avg_10,median_area_avg_10,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,017,1.0,1.5,0.0,0.0,2.0,5095.450,6.20,468.17,230.13,164.19
1,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,034,1.0,2.0,11.0,3000.0,9.0,38291.770,0.50,122.27,NaN,NaN
2,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,039,1.0,2.0,1.0,2500.0,11.0,42910.990,3.83,314.67,214.37,164.46
3,05680,2002,1,1.0,0.0,0.0,2.0,11377.32,056,2.0,1.5,0.0,0.0,2.0,6344.380,5.12,105.12,NaN,NaN
4,06023,1992,1,1.0,0.0,250.0,1.0,4609.14,060,1.0,1.0,0.0,250.0,1.0,4609.140,NaN,100.00,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,99669,2002,1,2.0,0.0,0.0,4.0,13912.96,996,2.0,2.0,0.0,150.0,16.0,32151.255,1.72,116.20,NaN,107.77
208,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,997,1.0,1.0,2.0,800.0,4.0,22130.060,20.76,72.44,120.76,69.72
209,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,997,3.0,1.0,0.0,0.0,4.0,1681.670,4.43,113.15,188.63,108.90
210,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,997,4.0,1.0,0.0,0.0,4.0,4200.325,2.88,246.91,162.72,110.30


In [ ]:
original_data_10.columns

Index(['zipcode', 'Year', 'flood_count', 'max_severity', 'median_dead',
       'median_displaced', 'median_duration', 'median_area', 'zipcode_3area',
       'flood_count_total_10', 'max_severity_avg_10', 'median_dead_avg_10',
       'median_displaced_avg_10', 'median_duration_avg_10',
       'median_area_avg_10', 'Annual Change (%)', 'HPI', 'HPI with 1990 base',
       'HPI with 2000 base'],
      dtype='object')

In [ ]:
from sklearn.mixture import GaussianMixture

from sklearn.metrics import pairwise_distances_argmin_min

numeric_columns = ['flood_count', 'max_severity', 'median_dead',
       'median_displaced', 'median_duration', 'median_area',
       'flood_count_total_10', 'max_severity_avg_10', 'median_dead_avg_10',
       'median_displaced_avg_10', 'median_duration_avg_10',
       'median_area_avg_10', 'HPI']
data_for_gmm_10 = original_data_10[numeric_columns].dropna()

gmm_10 = GaussianMixture(n_components=3, random_state=42)
gmm_10.fit(data_for_gmm_10)

synthetic_data_10 = gmm_10.sample(n_samples=1000)
synthetic_df_10 = pd.DataFrame(synthetic_data_10[0], columns=numeric_columns)

distances = pairwise_distances_argmin_min(synthetic_df_10[numeric_columns], original_data_10[numeric_columns])
synthetic_df_10['zipcode'] = original_data_10['zipcode'].iloc[distances[0]].values
synthetic_df_10['Year'] = original_data_10['Year'].iloc[distances[0]].values

synthetic_df_10['zipcode_3area'] = synthetic_df_10['zipcode'].astype(str).str[:3]


expanded_data_10 = pd.concat([original_data_10, synthetic_df_10], axis=0)
expanded_data_10 = expanded_data_10.reset_index(drop=True)



expanded_data_10

/usr/local/lib/python3.11/dist-packages/sklearn/mixture/_base.py:436: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,zipcode_3area,flood_count_total_10,max_severity_avg_10,median_dead_avg_10,median_displaced_avg_10,median_duration_avg_10,median_area_avg_10,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1.000000,1.500000,0.000000,0.000000e+00,2.000000,5095.450000,017,1.000000,1.500000,0.000000,0.000000e+00,2.000000,5095.450000,6.20,468.170000,230.13,164.19
1,03448,2005,1.000000,2.000000,11.000000,3.000000e+03,9.000000,38291.770000,034,1.000000,2.000000,11.000000,3.000000e+03,9.000000,38291.770000,0.50,122.270000,NaN,NaN
2,03903,2006,1.000000,2.000000,1.000000,2.500000e+03,11.000000,42910.990000,039,1.000000,2.000000,1.000000,2.500000e+03,11.000000,42910.990000,3.83,314.670000,214.37,164.46
3,05680,2002,1.000000,1.000000,0.000000,0.000000e+00,2.000000,11377.320000,056,2.000000,1.500000,0.000000,0.000000e+00,2.000000,6344.380000,5.12,105.120000,NaN,NaN
4,06023,1992,1.000000,1.000000,0.000000,2.500000e+02,1.000000,4609.140000,060,1.000000,1.000000,0.000000,2.500000e+02,1.000000,4609.140000,NaN,100.000000,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,28305,2018,1.001068,1.999533,12.001162,1.500000e+06,17.000652,95203.219275,283,0.999805,2.002053,12.001409,1.500000e+06,17.000060,95203.221170,NaN,262.810327,NaN,NaN
1208,28305,2018,1.001194,2.001770,12.001184,1.500000e+06,17.000883,95203.218883,283,0.999889,2.000831,11.998089,1.500000e+06,16.999960,95203.220234,NaN,262.810477,NaN,NaN
1209,28305,2018,0.999446,1.999217,12.001910,1.500000e+06,17.001170,95203.218431,283,1.000204,1.999548,12.000787,1.500000e+06,16.999689,95203.221915,NaN,262.808858,NaN,NaN
1210,28305,2018,0.999677,2.000364,11.999050,1.500000e+06,17.000830,95203.217737,283,0.998296,2.001391,11.999991,1.500000e+06,16.998227,95203.219881,NaN,262.811521,NaN,NaN


In [ ]:
X = expanded_data_10[['flood_count', 'max_severity', 'median_dead', 'median_displaced',
                 'median_duration', 'median_area',
                 'flood_count_total_10', 'max_severity_avg_10',
                 'median_dead_avg_10', 'median_displaced_avg_10',
                 'median_duration_avg_10', 'median_area_avg_10', 'zipcode_3area']]

y = expanded_data_10['HPI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_split= 15, min_samples_leaf= 1, max_depth= 40, bootstrap= True)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

# evaluate the model performance
print("Random Forest Regressor: Predicting HPI(w/ past history, prior 10 year flood averages, synthetic data):")
r2 = r2_score(y_test, y_pred)
print(f'R² score: {r2:.3f}')


mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae:.3f}')

relative_error = mae / np.mean(y_test) * 100
print(f'Relative Error: {relative_error:.2f}%')



print(f"Predictions: {y_pred[:10]}")
print(f"Actual: {y_test[:10].values}")

Random Forest Regressor: Predicting HPI(w/ past history, prior 10 year flood averages, synthetic data):
R² score: 0.112
Mean Absolute Error (MAE): 115.054
Relative Error: 48.46%
Predictions: [183.8093634  137.34371065 221.12881285 170.53125115 208.36529259
 135.52214951 184.25855859 244.53747956 303.18762199 223.72528168]
Actual: [111.83751463  87.91973092 162.32       320.37251149 139.71
 168.44417393 373.07095555 246.91       263.55377839  90.48598224]


#TabPFN

###Install package

In [2]:
#https://github.com/PriorLabs/TabPFN?tab=readme-ov-file
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

## Real Housing-Flood Data

In [32]:
import pandas as pd
import numpy as np
df_real = pd.read_csv('merged_housing_flood_data.csv', dtype={'zipcode': str})
df_real

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,6.20,468.17,230.13,164.19
1,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,0.50,122.27,NaN,NaN
2,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,3.83,314.67,214.37,164.46
3,05680,2002,1,1.0,0.0,0.0,2.0,11377.32,5.12,105.12,NaN,NaN
4,06023,1992,1,1.0,0.0,250.0,1.0,4609.14,NaN,100.00,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...
207,99669,2002,1,2.0,0.0,0.0,4.0,13912.96,1.72,116.20,NaN,107.77
208,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,20.76,72.44,120.76,69.72
209,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,4.43,113.15,188.63,108.90
210,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,2.88,246.91,162.72,110.30


In [33]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# Assuming there is a TabPFNRegressor (if not, a different regressor should be used)
from tabpfn import TabPFNRegressor

X = df_real[['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area']]
y = df_real['HPI']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

# Predict on the test set
predictions = regressor.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
relative_error = mae / np.mean(y_test) * 100

print("TabPFNRegressor (current year only, real data)")
print(f'Mean Absolute Error (MAE): {mae:.3f}')
print("R² Score:", r2)

print(f'Relative Error: {relative_error:.2f}%')

TabPFNRegressor (current year only, real data)
Mean Absolute Error (MAE): 114.973
R² Score: 0.0024927002698592604
Relative Error: 46.99%


In [34]:
df_real_10_avg = pd.read_csv('merged_housing_flood_data_avg_10_years.csv', dtype={'zipcode': str})
df_real_10_avg

,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,zipcode_3area,flood_count_total_10,max_severity_avg_10,median_dead_avg_10,median_displaced_avg_10,median_duration_avg_10,median_area_avg_10,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1,1.5,0.0,0.0,2.0,5095.45,17,1.0,1.5,0.0,0.0,2.0,5095.450,6.20,468.17,230.13,164.19
1,03448,2005,1,2.0,11.0,3000.0,9.0,38291.77,34,1.0,2.0,11.0,3000.0,9.0,38291.770,0.50,122.27,NaN,NaN
2,03903,2006,1,2.0,1.0,2500.0,11.0,42910.99,39,1.0,2.0,1.0,2500.0,11.0,42910.990,3.83,314.67,214.37,164.46
3,05680,2002,1,1.0,0.0,0.0,2.0,11377.32,56,2.0,1.5,0.0,0.0,2.0,6344.380,5.12,105.12,NaN,NaN
4,06023,1992,1,1.0,0.0,250.0,1.0,4609.14,60,1.0,1.0,0.0,250.0,1.0,4609.140,NaN,100.00,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,99669,2002,1,2.0,0.0,0.0,4.0,13912.96,996,2.0,2.0,0.0,150.0,16.0,32151.255,1.72,116.20,NaN,107.77
208,99705,1991,1,1.0,2.0,800.0,4.0,22130.06,997,1.0,1.0,2.0,800.0,4.0,22130.060,20.76,72.44,120.76,69.72
209,99705,2003,1,1.0,0.0,0.0,4.0,1681.67,997,3.0,1.0,0.0,0.0,4.0,1681.670,4.43,113.15,188.63,108.90
210,99709,2003,1,1.0,0.0,0.0,5.0,6718.98,997,4.0,1.0,0.0,0.0,4.0,4200.325,2.88,246.91,162.72,110.30


In [35]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# Assuming there is a TabPFNRegressor (if not, a different regressor should be used)
from tabpfn import TabPFNRegressor

X = df_real_10_avg[['flood_count', 'max_severity', 'median_dead', 'median_displaced',
                 'median_duration', 'median_area',
                 'flood_count_total_10', 'max_severity_avg_10',
                 'median_dead_avg_10', 'median_displaced_avg_10',
                 'median_duration_avg_10', 'median_area_avg_10']]
y = df_real_10_avg['HPI']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

# Predict on the test set
predictions = regressor.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
relative_error = mae / np.mean(y_test) * 100

print("TabPFNRegressor (including 10 year prior averages, real data)")
print(f'Mean Absolute Error (MAE): {mae:.3f}')
print("R² Score:", r2)
print(f'Relative Error: {relative_error:.2f}%')

TabPFNRegressor (including 10 year prior averages, real data)
Mean Absolute Error (MAE): 126.754
R² Score: 0.05903686284911025
Relative Error: 51.29%


## Synthetic Housing-Flood Data

### Current Year Data

In [44]:
from sklearn.mixture import GaussianMixture

from sklearn.metrics import pairwise_distances_argmin_min

numeric_columns = ['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area', 'HPI']
data_for_gmm = df_real[numeric_columns].dropna()

gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(data_for_gmm)

synthetic_data = gmm.sample(n_samples=800)
synthetic_df = pd.DataFrame(synthetic_data[0], columns=numeric_columns)

distances = pairwise_distances_argmin_min(synthetic_df[numeric_columns], df_real[numeric_columns])
synthetic_df['zipcode'] = df_real['zipcode'].iloc[distances[0]].values
synthetic_df['Year'] = df_real['Year'].iloc[distances[0]].values

expanded_data_tabpfn = pd.concat([df_real, synthetic_df], axis=0)
expanded_data_tabpfn = expanded_data_tabpfn.reset_index(drop=True)


expanded_data_tabpfn

/usr/local/lib/python3.11/dist-packages/sklearn/mixture/_base.py:436: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1.000000,1.500000,0.000000,0.000000e+00,2.000000,5095.450000,6.20,468.170000,230.13,164.19
1,03448,2005,1.000000,2.000000,11.000000,3.000000e+03,9.000000,38291.770000,0.50,122.270000,NaN,NaN
2,03903,2006,1.000000,2.000000,1.000000,2.500000e+03,11.000000,42910.990000,3.83,314.670000,214.37,164.46
3,05680,2002,1.000000,1.000000,0.000000,0.000000e+00,2.000000,11377.320000,5.12,105.120000,NaN,NaN
4,06023,1992,1.000000,1.000000,0.000000,2.500000e+02,1.000000,4609.140000,NaN,100.000000,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...
1007,28305,2018,1.000234,2.001134,12.001295,1.500000e+06,16.999160,95203.222220,NaN,262.809618,NaN,NaN
1008,28305,2018,0.998840,2.000257,11.999744,1.500000e+06,17.000948,95203.219200,NaN,262.810502,NaN,NaN
1009,28305,2018,1.000618,2.000630,11.999525,1.500000e+06,16.998556,95203.221580,NaN,262.810486,NaN,NaN
1010,28305,2018,0.999143,2.000091,12.000222,1.500000e+06,17.000769,95203.219620,NaN,262.808616,NaN,NaN


In [45]:
X = expanded_data_tabpfn[['flood_count', 'max_severity', 'median_dead', 'median_displaced', 'median_duration', 'median_area']]
y = expanded_data_tabpfn['HPI']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

# Predict on the test set
predictions = regressor.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
relative_error = mae / np.mean(y_test) * 100

print("TabPFNRegressor (current year only, synthetic data)")
print(f'Mean Absolute Error (MAE): {mae:.3f}')
print("R² Score:", r2)
print(f'Relative Error: {relative_error:.2f}%')

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_data.py:3453: RuntimeWarning: overflow encountered in cast
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_data.py:3453: RuntimeWarning: overflow encountered in cast
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_data.py:3453: RuntimeWarning: overflow encountered in cast
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_data.py:3453: RuntimeWarning: overflow encountered in cast
  x_inv[pos] = np.power(x[pos] * lmbda + 1, 1 / lmbda) - 1


TabPFNRegressor (current year only, synthetic data)
Mean Absolute Error (MAE): 85.153
R² Score: 0.4012760482025821
Relative Error: 37.35%


### Prior 10 year averages

In [36]:
numeric_columns = ['flood_count', 'max_severity', 'median_dead',
       'median_displaced', 'median_duration', 'median_area',
       'flood_count_total_10', 'max_severity_avg_10', 'median_dead_avg_10',
       'median_displaced_avg_10', 'median_duration_avg_10',
       'median_area_avg_10', 'HPI']
data_for_gmm_10 = df_real_10_avg[numeric_columns].dropna()

gmm_10 = GaussianMixture(n_components=3, random_state=42)
gmm_10.fit(data_for_gmm_10)

synthetic_data_10 = gmm_10.sample(n_samples=800)
synthetic_df_10 = pd.DataFrame(synthetic_data_10[0], columns=numeric_columns)

distances = pairwise_distances_argmin_min(synthetic_df_10[numeric_columns], df_real_10_avg[numeric_columns])
synthetic_df_10['zipcode'] = df_real_10_avg['zipcode'].iloc[distances[0]].values
synthetic_df_10['Year'] = df_real_10_avg['Year'].iloc[distances[0]].values
synthetic_df_10['zipcode_3area'] = synthetic_df_10['zipcode'].astype(str).str[:3]

expanded_data_tabpfn_10 = pd.concat([df_real_10_avg, synthetic_df_10], axis=0)
expanded_data_tabpfn_10 = expanded_data_tabpfn_10.reset_index(drop=True)


expanded_data_tabpfn_10

/usr/local/lib/python3.11/dist-packages/sklearn/mixture/_base.py:436: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


,zipcode,Year,flood_count,max_severity,median_dead,median_displaced,median_duration,median_area,zipcode_3area,flood_count_total_10,max_severity_avg_10,median_dead_avg_10,median_displaced_avg_10,median_duration_avg_10,median_area_avg_10,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,01749,2018,1.000000,1.500000,0.000000,0.000000e+00,2.000000,5095.450000,17,1.000000,1.500000,0.000000,0.000000e+00,2.000000,5095.450000,6.20,468.170000,230.13,164.19
1,03448,2005,1.000000,2.000000,11.000000,3.000000e+03,9.000000,38291.770000,34,1.000000,2.000000,11.000000,3.000000e+03,9.000000,38291.770000,0.50,122.270000,NaN,NaN
2,03903,2006,1.000000,2.000000,1.000000,2.500000e+03,11.000000,42910.990000,39,1.000000,2.000000,1.000000,2.500000e+03,11.000000,42910.990000,3.83,314.670000,214.37,164.46
3,05680,2002,1.000000,1.000000,0.000000,0.000000e+00,2.000000,11377.320000,56,2.000000,1.500000,0.000000,0.000000e+00,2.000000,6344.380000,5.12,105.120000,NaN,NaN
4,06023,1992,1.000000,1.000000,0.000000,2.500000e+02,1.000000,4609.140000,60,1.000000,1.000000,0.000000,2.500000e+02,1.000000,4609.140000,NaN,100.000000,NaN,94.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,28305,2018,1.000900,1.999301,11.999990,1.500000e+06,17.000504,95203.219406,283,0.998974,1.999557,12.001409,1.500000e+06,16.999460,95203.220073,NaN,262.810025,NaN,NaN
1008,28305,2018,0.999521,1.999724,12.000267,1.500000e+06,17.001927,95203.222768,283,0.998668,1.999796,11.999833,1.500000e+06,17.002623,95203.221408,NaN,262.810016,NaN,NaN
1009,28305,2018,0.999047,1.999231,12.000472,1.500000e+06,17.000786,95203.219713,283,1.000615,2.000428,12.001876,1.500000e+06,17.000321,95203.219035,NaN,262.809508,NaN,NaN
1010,28305,2018,0.999056,2.001335,11.999845,1.500000e+06,16.999540,95203.221012,283,0.999328,1.998556,11.999746,1.500000e+06,16.998892,95203.221442,NaN,262.811189,NaN,NaN


In [46]:
X = expanded_data_tabpfn_10[['flood_count', 'max_severity', 'median_dead',
       'median_displaced', 'median_duration', 'median_area',
       'flood_count_total_10', 'max_severity_avg_10', 'median_dead_avg_10',
       'median_displaced_avg_10', 'median_duration_avg_10',
       'median_area_avg_10']]
y = expanded_data_tabpfn_10['HPI']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the regressor
regressor = TabPFNRegressor()
regressor.fit(X_train, y_train)

# Predict on the test set
predictions = regressor.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
relative_error = mae / np.mean(y_test) * 100

print("TabPFNRegressor (prior 10 year averages, synthetic data)")
print(f'Mean Absolute Error (MAE): {mae:.3f}')
print("R² Score:", r2)
print(f'Relative Error: {relative_error:.2f}%')

TabPFNRegressor (prior 10 year averages, synthetic data)
Mean Absolute Error (MAE): 113.451
R² Score: 0.17103261874091413
Relative Error: 49.87%
